In [ ]:
# Step 1: Install required libraries
!pip install transformers torch librosa deepai googletrans==4.0.0-rc1 --user

  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached deepai-1.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached hstspreload-2024.4.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached scipy-1.13.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.4.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.4.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached numba-0.59.1-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata (14 kB)
  Using cached pooch-1.8.1-py3-none-any.whl.metadata (9.5 kB)
  Using 

In [2]:
# Import libraries
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa

from googletrans import Translator
import deepai
import requests

In [20]:
# Step 4: Load the Hugging Face model
processor = Wav2Vec2Processor.from_pretrained("bayartsogt/wav2vec2-large-xlsr-mongolian")
model = Wav2Vec2ForCTC.from_pretrained("bayartsogt/wav2vec2-large-xlsr-mongolian")

Some weights of the model checkpoint at bayartsogt/wav2vec2-large-xlsr-mongolian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at bayartsogt/wav2vec2-large-xlsr-mongolian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should 

In [17]:
# Step 5: Read and process audio file
def speech_to_text(audio_file):
    speech, rate = librosa.load(audio_file, sr=16000)
    input_values = processor(speech, return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

mongolian_text = speech_to_text('library.wav')
print("Mongolian Text:", mongolian_text)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Mongolian Text: номын бсан


In [13]:
# Step 6: Translate Mongolian text to English
translator = Translator()
translated_text = translator.translate(mongolian_text, src='mn', dest='en').text
print("Translated Text (English):", translated_text)


Translated Text (English): The books of books


In [14]:
# Replace 'your_deepai_api_key' with your actual DeepAI API key
deepai.api_key = '3f7170f7-294d-4358-b1f3-5691dc159aee'

# Example of API call with the key
response = requests.post(
    "https://api.deepai.org/api/text2img",
    data={'text': translated_text},
    headers={'api-key': '3f7170f7-294d-4358-b1f3-5691dc159aee'}  # Replace this placeholder
)


# Add this code after the API call to print the full response
complete_response = response.json()
print("Full Response:", complete_response)

# Check if there's an 'error' key in the response
if 'error' in complete_response:
    print("Error Message:", complete_response['error'])
else:
    image_url = complete_response.get('output_url')
    print("Generated Image URL:", image_url)



Full Response: {'id': '4132afeb-2239-40a9-85f6-cadbe504e5d8', 'output_url': 'https://api.deepai.org/job-view-file/4132afeb-2239-40a9-85f6-cadbe504e5d8/outputs/output.jpg'}
Generated Image URL: https://api.deepai.org/job-view-file/4132afeb-2239-40a9-85f6-cadbe504e5d8/outputs/output.jpg
